In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader
from torch.utils.data.dataset import Dataset
import os
from google.colab import drive

# Montar o Google Drive
drive.mount('/content/drive')

print(os.listdir(('/content/drive/MyDrive/compara_segmentadores/data2/talho/G1020/train/images/')))

Mounted at /content/drive
['class 0', 'class 1']


In [ ]:
!pip install torch torchvision opencv-python
!pip install scikit-plot
!pip install django-model-utils

In [ ]:
pip install timm

In [3]:
#SENet
modelo = "SENet"

import os
import torch
from torch import nn, optim
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from sklearn.metrics import confusion_matrix, classification_report, roc_curve, roc_auc_score, precision_recall_curve, accuracy_score, precision_score, recall_score, f1_score
import seaborn as sn
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import timm

# Função para carregar as imagens e anotações
def load_data(root_dir):
    image_dir_train = os.path.join(root_dir, 'train/images')
    annotation_dir_train = os.path.join(root_dir, 'train/anotacoes')


    transform = transforms.Compose([
        transforms.Resize((224, 224)),  # Redimensionar todas as imagens para o tamanho 224x224
        transforms.ToTensor()
    ])

    train_data = torchvision.datasets.ImageFolder(
        root=image_dir_train,
        transform=transform
    )
    train_loader = torch.utils.data.DataLoader(
        train_data,
        batch_size=16,
        shuffle=True
    )
    return train_loader

def test_load_data(root_dir):
    image_dir_test = os.path.join(root_dir, 'test/images')
    annotation_dir_test = os.path.join(root_dir, 'test/anotacoes')

    transform = transforms.Compose([
        transforms.Resize((224, 224)),  # Redimensionar todas as imagens para o tamanho 224x224
        transforms.ToTensor()
    ])
    test_data = torchvision.datasets.ImageFolder(
        root=image_dir_test,
        transform=transform
    )
    test_loader = torch.utils.data.DataLoader(
        test_data,
        batch_size=16,
        shuffle=False
    )

    return test_loader

def evaluate_model(model, data_loader):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)
    model.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for inputs, labels in data_loader:
            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)

            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = correct / total
    return accuracy

def validation_load_data(root_dir):  # Corrigir o nome do parâmetro para root_dir
    image_dir_val = os.path.join(root_dir, 'valid/images')
    annotation_dir_val = os.path.join(root_dir, 'valid/anotacoes')

    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor()
    ])

    val_data = torchvision.datasets.ImageFolder(
        root=image_dir_val,
        transform=transform
    )
    val_loader = torch.utils.data.DataLoader(
        val_data,
        batch_size=16,
        shuffle=False
    )

    return val_loader

def train_model(train_loader, val_loader, num_epochs,MODELO):
    model = timm.create_model('seresnext26d_32x4d', pretrained=True)  # Use a arquitetura SENet, por exemplo, 'senet154'
    num_ftrs = model.fc.in_features
    model.fc = nn.Linear(num_ftrs, 2)

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

    best_accuracy = 0.0  # Track the best test accuracy
    train_accuracies = []  # List to store train accuracies
    train_losses = []  # List to store train losses

    best_val_accuracy = 0.0  # Track the best validation accuracy
    val_accuracies = []  # List to store validation accuracies



    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0

        for inputs, labels in train_loader:
            inputs = inputs.to(device)
            labels = labels.to(device)

            optimizer.zero_grad()

            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        epoch_loss = running_loss / len(train_loader)
        train_losses.append(epoch_loss)

        train_accuracy = evaluate_model(model, train_loader)
        train_accuracies.append(train_accuracy)

        # Validation after each epoch
        model.eval()
        val_accuracy = evaluate_model(model, val_loader)
        val_accuracies.append(val_accuracy)

        print(f'Epoch [{epoch+1}/{num_epochs}], Train Accuracy: {train_accuracy:.2%}, Validation Accuracy: {val_accuracy:.2%}, Loss: {epoch_loss:.4f}')

        #aux = round(val_accuracy,2)
        # Save the best model based on the highest validation accuracy
        #if aux != 0.83 or epoch == num_epochs-1:
        #       if val_accuracy > best_accuracy:
        #         best_accuracy = val_accuracy
        #         print("best_accuracy:", best_accuracy,)
        #         print("val_accuracy:", val_accuracy)
        #         print("aux:", aux)
        #else:
        #       print("best_accuracy:", best_accuracy,)

        # Save the best model based on the highest validation accuracy
        if val_accuracy > best_accuracy:
            best_accuracy = val_accuracy
            best_model_state = model.state_dict()

    # Save the best model checkpoint
    torch.save(best_model_state, MODELO)

    return model


def test_model(model, test_loader):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")  # Define o dispositivo de execução
    model.eval()
    y_true = []
    y_pred = []
    y_scores = []

    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)

            y_true.extend(labels.cpu().numpy())
            y_pred.extend(predicted.cpu().numpy())

            # Obter as probabilidades de saída (scores)
            scores = F.softmax(outputs, dim=1)
            y_scores.extend(scores[:, 1].cpu().numpy())

    # Calcula a matriz de confusão
    cm = confusion_matrix(y_true, y_pred)
    tn, fp, fn, tp = cm.ravel()
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)

    print("Acurácia:", accuracy)
    print("Precisão:", precision)
    print("Recall:", recall)
    print("F1-score:", f1)

    print("True Negative (TN):", tn)
    print("False Positive (FP):", fp)
    print("False Negative (FN):", fn)
    print("True Positive (TP):", tp)
    class_names = ['Classe 0 (Sem Glaucoma)', 'Classe 1 (Com Glaucoma)']

    # Plot da matriz de confusão
    plt.figure(figsize=(8, 6))
    plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
    plt.title('Matriz de Confusão')
    plt.colorbar()
    tick_marks = np.arange(len(class_names))
    plt.xticks(tick_marks, class_names, rotation=45)
    plt.yticks(tick_marks, class_names)
    plt.xlabel('Predito')
    plt.ylabel('Verdadeiro')
    plt.show()

    # Relatório de classificação
    print("Relatório de Classificação:")
    print(classification_report(y_true, y_pred, target_names=class_names))

    # Calcular as estatísticas ROC
    fpr, tpr, thresholds_roc = roc_curve(y_true, y_scores)
    roc_auc = roc_auc_score(y_true, y_scores)

    # Plot do gráfico ROC
    plt.figure(figsize=(8, 6))
    plt.plot(fpr, tpr, label='Curva ROC (AUC = {:.2f})'.format(roc_auc))
    plt.plot([0, 1], [0, 1], 'k--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('Taxa de Falso Positivo')
    plt.ylabel('Taxa de Verdadeiro Positivo')
    plt.title('Gráfico ROC')
    plt.legend(loc='lower right')
    plt.show()

    # Calcular as estatísticas RC
    precision, recall, thresholds_rc = precision_recall_curve(y_true, y_scores)

    # Plot do gráfico RC
    plt.figure(figsize=(8, 6))
    plt.plot(recall, precision, label='Curva RC')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('Recall')
    plt.ylabel('Precision')
    plt.title('Gráfico RC')
    plt.legend(loc='lower left')
    plt.show()

    # Relatório de classificação
    print("Relatório de Classificação:")
    print(classification_report(y_true, y_pred, target_names=class_names))


Valid_root = "/content/drive/MyDrive/compara_segmentadores/data2/talho/ORIGA/"
Teste_root = "/content/drive/MyDrive/compara_segmentadores/data2/talho/G1020/"
num_epochs = 3

Val_loader = validation_load_data(Valid_root)
Test_loader = test_load_data(Teste_root)

print('\n')
# Carregar os dados para G1020
print("G1020:")
TrainG1020_root = "/content/drive/MyDrive/compara_segmentadores/data2/talho/G1020/"
Train_loader_G1020 = load_data(TrainG1020_root)
G1020M = 'G1020best_model.pth'
MG1020M = modelo+G1020M
modelo_G1020 = train_model(Train_loader_G1020, Val_loader, num_epochs,MG1020M)
print("Fim do treino.")
melhor_estado_modelo_G1020 = torch.load(MG1020M)
modelo_G1020.load_state_dict(melhor_estado_modelo_G1020)
test_model(modelo_G1020, Test_loader)
print("Fim do teste.")
print('\n')

print('\n')
# Carregar os dados para ORIGA
print("ORIGA:")
TrainORIGA_root = "/content/drive/MyDrive/compara_segmentadores/data2/talho/ORIGA/"
train_loader_ORIGA = load_data(TrainORIGA_root)
ORIGM = 'ORIGAbest_model.pth'
MORIGM = modelo+ORIGM
modelo_ORIGA = train_model(train_loader_ORIGA, Val_loader, num_epochs,MORIGM)
print("Fim do treino.")
melhor_estado_modelo_ORIGA = torch.load(MORIGM)
modelo_ORIGA.load_state_dict(melhor_estado_modelo_ORIGA)
test_model(modelo_ORIGA, Test_loader)
print("Fim do teste.")
print('\n')

print('\n')
# Carregar os dados para REFUGE
print("REFUGE:")
TrainREFUGE_root = "/content/drive/MyDrive/compara_segmentadores/data2/talho/REFUGE/"
train_loader_REFUGE = load_data(TrainREFUGE_root)
REFUM = 'REFUGEbest_model.pth'
MREFUM = modelo+REFUM
modelo_REFUGE = train_model(train_loader_REFUGE, Val_loader, num_epochs,MREFUM)
print("Fim do treino.")
melhor_estado_modelo_REFUGE = torch.load(MREFUM)
modelo_REFUGE.load_state_dict(melhor_estado_modelo_REFUGE)
test_model(modelo_REFUGE, Test_loader)
print("Fim do teste.")
print('\n')


NameError: ignored

In [ ]:
#SENet
modelo = "SENet"

import os
import torch
import torchvision
from torch import nn, optim
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from sklearn.metrics import confusion_matrix, classification_report, roc_curve, roc_auc_score, precision_recall_curve, accuracy_score, precision_score, recall_score, f1_score
import seaborn as sn
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import timm


# Função para carregar as imagens e anotações
def load_data(root_dir):
    image_dir_train = os.path.join(root_dir, 'train/images')
    annotation_dir_train = os.path.join(root_dir, 'train/anotacoes')


    transform = transforms.Compose([
        transforms.Resize((224, 224)),  # Redimensionar todas as imagens para o tamanho 224x224
        transforms.ToTensor()
    ])

    train_data = torchvision.datasets.ImageFolder(
        root=image_dir_train,
        transform=transform
    )
    train_loader = torch.utils.data.DataLoader(
        train_data,
        batch_size=16,
        shuffle=True
    )
    return train_loader

def test_load_data(root_dir):
    image_dir_test = os.path.join(root_dir, 'test/images')
    annotation_dir_test = os.path.join(root_dir, 'test/anotacoes')

    transform = transforms.Compose([
        transforms.Resize((224, 224)),  # Redimensionar todas as imagens para o tamanho 224x224
        transforms.ToTensor()
    ])
    test_data = torchvision.datasets.ImageFolder(
        root=image_dir_test,
        transform=transform
    )
    test_loader = torch.utils.data.DataLoader(
        test_data,
        batch_size=16,
        shuffle=False
    )

    return test_loader

def evaluate_model(model, data_loader):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)
    model.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for inputs, labels in data_loader:
            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)

            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = correct / total
    return accuracy

def validation_load_data(root_dir):  # Corrigir o nome do parâmetro para root_dir
    image_dir_val = os.path.join(root_dir, 'valid/images')
    annotation_dir_val = os.path.join(root_dir, 'valid/anotacoes')

    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor()
    ])

    val_data = torchvision.datasets.ImageFolder(
        root=image_dir_val,
        transform=transform
    )
    val_loader = torch.utils.data.DataLoader(
        val_data,
        batch_size=16,
        shuffle=False
    )

    return val_loader

def train_model(train_loader, val_loader, num_epochs,MODELO):
    model = timm.create_model('seresnet101')  # Use a arquitetura SENet, por exemplo, 'senet154'
    num_ftrs = model.fc.in_features
    model.fc = nn.Linear(num_ftrs, 2)

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

    best_accuracy = 0.0  # Track the best test accuracy
    train_accuracies = []  # List to store train accuracies
    train_losses = []  # List to store train losses

    best_val_accuracy = 0.0  # Track the best validation accuracy
    val_accuracies = []  # List to store validation accuracies



    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0

        for inputs, labels in train_loader:
            inputs = inputs.to(device)
            labels = labels.to(device)

            optimizer.zero_grad()

            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        epoch_loss = running_loss / len(train_loader)
        train_losses.append(epoch_loss)

        train_accuracy = evaluate_model(model, train_loader)
        train_accuracies.append(train_accuracy)

        # Validation after each epoch
        model.eval()
        val_accuracy = evaluate_model(model, val_loader)
        val_accuracies.append(val_accuracy)

        print(f'Epoch [{epoch+1}/{num_epochs}], Train Accuracy: {train_accuracy:.2%}, Validation Accuracy: {val_accuracy:.2%}, Loss: {epoch_loss:.4f}')

        #aux = round(val_accuracy,2)
        # Save the best model based on the highest validation accuracy
        #if aux != 0.83 or epoch == num_epochs-1:
        #       if val_accuracy > best_accuracy:
        #         best_accuracy = val_accuracy
        #         print("best_accuracy:", best_accuracy,)
        #         print("val_accuracy:", val_accuracy)
        #         print("aux:", aux)
        #else:
        #       print("best_accuracy:", best_accuracy,)

        # Save the best model based on the highest validation accuracy
        if val_accuracy > best_accuracy:
            best_accuracy = val_accuracy
            best_model_state = model.state_dict()

    # Save the best model checkpoint
    torch.save(best_model_state, MODELO)

    return model


def test_model(model, test_loader):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")  # Define o dispositivo de execução
    model.eval()
    y_true = []
    y_pred = []
    y_scores = []

    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)

            y_true.extend(labels.cpu().numpy())
            y_pred.extend(predicted.cpu().numpy())

            # Obter as probabilidades de saída (scores)
            scores = F.softmax(outputs, dim=1)
            y_scores.extend(scores[:, 1].cpu().numpy())

    # Calcula a matriz de confusão
    cm = confusion_matrix(y_true, y_pred)
    tn, fp, fn, tp = cm.ravel()
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)

    print("Acurácia:", accuracy)
    print("Precisão:", precision)
    print("Recall:", recall)
    print("F1-score:", f1)

    print("True Negative (TN):", tn)
    print("False Positive (FP):", fp)
    print("False Negative (FN):", fn)
    print("True Positive (TP):", tp)
    class_names = ['Classe 0 (Sem Glaucoma)', 'Classe 1 (Com Glaucoma)']

    # Plot da matriz de confusão
    plt.figure(figsize=(8, 6))
    plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
    plt.title('Matriz de Confusão')
    plt.colorbar()
    tick_marks = np.arange(len(class_names))
    plt.xticks(tick_marks, class_names, rotation=45)
    plt.yticks(tick_marks, class_names)
    plt.xlabel('Predito')
    plt.ylabel('Verdadeiro')
    plt.show()

    # Relatório de classificação
    print("Relatório de Classificação:")
    print(classification_report(y_true, y_pred, target_names=class_names))

    # Calcular as estatísticas ROC
    fpr, tpr, thresholds_roc = roc_curve(y_true, y_scores)
    roc_auc = roc_auc_score(y_true, y_scores)

    # Plot do gráfico ROC
    plt.figure(figsize=(8, 6))
    plt.plot(fpr, tpr, label='Curva ROC (AUC = {:.2f})'.format(roc_auc))
    plt.plot([0, 1], [0, 1], 'k--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('Taxa de Falso Positivo')
    plt.ylabel('Taxa de Verdadeiro Positivo')
    plt.title('Gráfico ROC')
    plt.legend(loc='lower right')
    plt.show()

    # Calcular as estatísticas RC
    precision, recall, thresholds_rc = precision_recall_curve(y_true, y_scores)

    # Plot do gráfico RC
    plt.figure(figsize=(8, 6))
    plt.plot(recall, precision, label='Curva RC')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('Recall')
    plt.ylabel('Precision')
    plt.title('Gráfico RC')
    plt.legend(loc='lower left')
    plt.show()

    # Relatório de classificação
    print("Relatório de Classificação:")
    print(classification_report(y_true, y_pred, target_names=class_names))


Valid_root = "/content/drive/MyDrive/compara_segmentadores/data2/talho/ORIGA/"
Teste_root = "/content/drive/MyDrive/compara_segmentadores/data2/talho/G1020/"
num_epochs = 3

Val_loader = validation_load_data(Valid_root)
Test_loader = test_load_data(Teste_root)

print('\n')
# Carregar os dados para G1020
print("G1020:")
TrainG1020_root = "/content/drive/MyDrive/compara_segmentadores/data2/talho/G1020/"
Train_loader_G1020 = load_data(TrainG1020_root)
G1020M = 'G1020best_model.pth'
MG1020M = modelo+G1020M
modelo_G1020 = train_model(Train_loader_G1020, Val_loader, num_epochs,MG1020M)
print("Fim do treino.")
melhor_estado_modelo_G1020 = torch.load(MG1020M)
modelo_G1020.load_state_dict(melhor_estado_modelo_G1020)
test_model(modelo_G1020, Test_loader)
print("Fim do teste.")
print('\n')

print('\n')
# Carregar os dados para ORIGA
print("ORIGA:")
TrainORIGA_root = "/content/drive/MyDrive/compara_segmentadores/data2/talho/ORIGA/"
train_loader_ORIGA = load_data(TrainORIGA_root)
ORIGM = 'ORIGAbest_model.pth'
MORIGM = modelo+ORIGM
modelo_ORIGA = train_model(train_loader_ORIGA, Val_loader, num_epochs,MORIGM)
print("Fim do treino.")
melhor_estado_modelo_ORIGA = torch.load(MORIGM)
modelo_ORIGA.load_state_dict(melhor_estado_modelo_ORIGA)
test_model(modelo_ORIGA, Test_loader)
print("Fim do teste.")
print('\n')

print('\n')
# Carregar os dados para REFUGE
print("REFUGE:")
TrainREFUGE_root = "/content/drive/MyDrive/compara_segmentadores/data2/talho/REFUGE/"
train_loader_REFUGE = load_data(TrainREFUGE_root)
REFUM = 'REFUGEbest_model.pth'
MREFUM = modelo+REFUM
modelo_REFUGE = train_model(train_loader_REFUGE, Val_loader, num_epochs,MREFUM)
print("Fim do treino.")
melhor_estado_modelo_REFUGE = torch.load(MREFUM)
modelo_REFUGE.load_state_dict(melhor_estado_modelo_REFUGE)
test_model(modelo_REFUGE, Test_loader)
print("Fim do teste.")
print('\n')




G1020:
Epoch [1/3], Train Accuracy: 71.37%, Validation Accuracy: 83.33%, Loss: 0.6343
Epoch [2/3], Train Accuracy: 71.37%, Validation Accuracy: 82.22%, Loss: 0.6177
Epoch [3/3], Train Accuracy: 67.51%, Validation Accuracy: 71.67%, Loss: 0.6153
Fim do treino.


NameError: ignored